In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install --quiet pyvis

In [ ]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
NEO4J_URL = "neo4j://localhost:7687"  # Replace with your server's URL
USERNAME = "neo4j"  # Replace with your username
PASSWORD = "your_password"  # Replace with your password

In [11]:
graph = Neo4jGraph(url=NEO4J_URL, username=USERNAME, password=PASSWORD)

In [4]:
loader = TextLoader(file_path="dummytext.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [32]:

llm = OllamaFunctions(model="qwen2:1.5b", temperature=0, format="json")

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)


In [ ]:
import networkx as nx
from pyvis.network import Network

# Create a directed graph
G = nx.Graph()

# Add nodes and edges from the existing data
for doc in graph_documents:
    for node in doc.nodes:
        G.add_node(node.id, label=node.id, title=node.type)
    for id_, rel in enumerate(doc.relationships):
        G.add_edge(rel.source.id, rel.target.id, label=rel.type)
      
# Create a Pyvis network
net = Network(notebook=True)

# Load the networkx graph into Pyvis
net.from_nx(G)

# Display the graph
net.show("graph.html")

In [34]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [40]:
graph_documents[0].nodes

[Node(id="Amico'S Family", type='Legacy', properties={}),
 Node(id='John Doe', type='Person', properties={})]

In [16]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

vector_index = Neo4jVector(embedding=embeddings,
    url=NEO4J_URL, 
    username=USERNAME, 
    password=PASSWORD)
vector_retriever = vector_index.as_retriever()

In [17]:
driver = GraphDatabase.driver(
        uri = NEO4J_URL,
        auth = (USERNAME, PASSWORD))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

Fulltext index created successfully.


In [35]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [36]:
entity_chain.invoke("Who are Nonna Lucia and Giovanni Caruso?")

Entities(names=['Nonna Lucia', 'Giovanni Caruso'])

In [20]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [37]:
print(graph_retriever("Who is Nonna Lucia?"))

Amico - TAUGHT -> Nonna Lucia
Lucia - WORKS_AT -> Los Angeles
Lucia - INHERITS_FROM -> grandmother
Lucia - SIBLINGS -> Antonio


In [22]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [23]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
chain.invoke(input="Who is Nonna Lucia? Did she teach anyone about restaurants or cooking?")

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.vector.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such vector schema index: vector}